In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)


In [2]:
from autogen_agentchat.agents import (AssistantAgent)

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client=model_client,
    system_message='You are a creating writer. Generate a short story about a brave knight and a dragon.'
)


In [3]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

Sir Cedric faced the mighty dragon, its scales shimmering like emeralds in the sun. With courage in his heart, he raised his sword, knowing the village depended on him. The dragon roared, but Cedric saw hope in its eyes. Maybe, they could be allies instead of enemies. And they were.


In [4]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants= [plot_agent, character_agent, ending_agent],
    max_turns=6
)
team.component_type


'team'

In [6]:
async def test_team():
    task = TextMessage(
        content='Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source='user'
    )

    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        

await test_team()


user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : In the heart of Verdant Forest, brave Alex and his loyal Pikachu venture to the hidden Mystic Cave. As shadows loom, they face a fierce Gengar. With courage and synergy, Pikachu's electrifying Thunderbolt pierces the darkness, revealing a treasure of ancient wisdom and forging their unbreakable bond.
character_writer : **Pokemon: Luna the Lucario**

Luna is a regal Lucario with a sleek, shimmering coat that reflects the moonlight. Her eyes, a deep azure, seem to hold the wisdom of ages. Trained in the mystical art of aura, she can sense the emotions and intentions of those around her, making her an invaluable ally. Luna's past is shrouded in mystery; she was found as an abandoned Riolu in the ancient Temple of Aura by Alex. Driven by loyalty and a deep bond with Alex, Luna's motivation is to protect and guide him, helping him become a formidable Pokemon Master. Her ultimate goal is to uncover t

In [7]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='db4f9ec5-3667-4db1-b91c-dfccd88c9f98' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 46, 14, 149301, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='a6ddf86d-4d6d-4bf0-b8fe-b794ebd1084d' source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=105) metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 46, 16, 159245, tzinfo=datetime.timezone.utc) content="Leaves of amber gently fall,  \nWhispering in the crisp, cool air's call.  \nUnder a sky of pumpkin gold,  \nNature's softest stories unfold.  \n\nA tapestry of rust and red,  \nBlankets the earth where creatures tread.  \nIn this magical, fleeting scene,  \nFall paints the world in shades serene.  \n\nThe harvest moon begins to rise,  \nReflecting in the woodland’s eyes.  \nAs seasons change in a gentle sweep,  \nThe world prepares for winter’s sleep.  \n" type='TextMessage'
id='08b9be3e-881f-4560-9de3

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat

team2 = RoundRobinGroupChat(
    participants= [plot_agent],
    max_turns=2
)


In [9]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team2.reset()  # Reset the team for a new task.
async for message in team2.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='1b7a16ce-df12-4b04-a7ae-ea680ee961c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 46, 55, 395616, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='bacaf9e1-e6da-46d3-be63-1db7d6575072' source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=72) metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 46, 57, 254543, tzinfo=datetime.timezone.utc) content="Leaves of amber softly fall,  \nUnderneath the trees so tall.  \nWhispering winds, a gentle call,  \nIn autumn’s golden, fleeting thrall.  \n\nPumpkin patches, fields of cheer,  \nCrisp air marks the turning year.  \nNature's canvas, clear and sheer,  \nEmbracing fall with warmth sincere.  " type='TextMessage'
id='d37c08f3-aefa-4a93-99a1-6293debd15a3' source='plot_writer' models_usage=RequestUsage(prompt_tokens=110, completion_tokens=353) metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 47, 2, 8

In [10]:
message


TaskResult(messages=[TextMessage(id='1b7a16ce-df12-4b04-a7ae-ea680ee961c5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 26, 15, 46, 55, 395616, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='bacaf9e1-e6da-46d3-be63-1db7d6575072', source='plot_writer', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=72), metadata={}, created_at=datetime.datetime(2025, 7, 26, 15, 46, 57, 254543, tzinfo=datetime.timezone.utc), content="Leaves of amber softly fall,  \nUnderneath the trees so tall.  \nWhispering winds, a gentle call,  \nIn autumn’s golden, fleeting thrall.  \n\nPumpkin patches, fields of cheer,  \nCrisp air marks the turning year.  \nNature's canvas, clear and sheer,  \nEmbracing fall with warmth sincere.  ", type='TextMessage'), TextMessage(id='d37c08f3-aefa-4a93-99a1-6293debd15a3', source='plot_writer', models_usage=RequestUsage(prompt_tokens=110, completion_tok

In [11]:
from autogen_agentchat.teams import RoundRobinGroupChat

team3 = RoundRobinGroupChat(
    participants= [plot_agent],
    # max_turns=2
)


In [ ]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team3.reset()  # Reset the team for a new task.
async for message in team3.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)


id='64b0bf80-62cf-45df-aaaa-77dec5cf8998' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 47, 12, 586281, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='edc1fb0e-f9d5-40d8-afd7-bc3a76223052' source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=68) metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 47, 14, 488692, tzinfo=datetime.timezone.utc) content="Leaves cascade in golden hues,  \nWhispering tales the wind pursues.  \nCrisp air dances through the trees,  \nAs harvest whispers in the breeze.  \n\nPumpkins glow with autumn's grace,  \nWhile shadows paint a cozy space.  \nNature dons its vibrant shawl,  \nIn the gentle, golden fall.  " type='TextMessage'
id='72c52757-5961-48ee-8273-00cf359485fd' source='plot_writer' models_usage=RequestUsage(prompt_tokens=106, completion_tokens=73) metadata={} created_at=datetime.datetime(2025, 7, 26, 15, 47, 16, 148